### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 2 - Digital Audio Processing 

In this assignment you will load, decode, and process digital audio files (e.g., MP3, WAV) using Python. For the following tasks, you will use our suggested libraries (see the setup section). For both audio formats you will extract and process content and some basic metadata. For the following tasks, you will use our suggested libraries (see the setup section). 

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments in Moodle.

If the cell below executes without error, you can start the assignment!

In [3]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
from enum import Enum
import mutagen      # mutagen
from mutagen.mp3 import MP3
from mutagen.id3 import ID3
from mutagen.easyid3 import EasyID3
import wave         # python's built-in wave library
import pandas as pd # pandas
import ffmpeg       # ffmpeg-python wrapper (requires ffmpeg.exe in your system path!)
import subprocess   # for calling local executables such as ffmpeg.exe


In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [4]:
# Please place your own imports here.
import numpy as np
import glob
from mutagen.wave import WAVE

In [ ]:
# Place any helper functions, global variables and classes here.

## Task 2.1 Organize Audio files by specific criteria (35P):

In [1087]:
# Write your function here.

# Auto-plays an audio file and also embeds an IPython audio display.
def MyAudioFilesOrganizer(inputDir: str, grouping) -> pd.DataFrame:
    columns = ['artist',
                'album',
                'genre',
                'format',
                'duration',
                'title',
                'date',
                'sample rate',
                'bitrate',
                'track',
                'composer',
                'encoder', 'channels']
  
    directory = glob.glob(inputDir)
    rows = []
    df = pd.DataFrame()
    
    for file in directory:
        with open(file, 'rb') as fl:
          
            flag = False
            isMpeg = False
            isFlac = False
            
            if fl.name.endswith('.mp3'):
                fileformat = 'mp3'
                isMpeg = True
                flag = True
            
            if fl.name.endswith('.flac'):
                fileformat = 'flac'
                isFlac = True
                flag = True
                
            elif fl.name.endswith('.wav') :
                fileformat = 'wav'
                
                #read file
                waveFile = WAVE(fl)
                
                # add new row to dataFrame
                rows = np.append(rows, fl.name)
                
                # get metadata
                title = "-"
                artist = "-"
                album = "-"
                genre = "-"
                duration = waveFile.info.length / 60
                bitrate = waveFile.info.bitrate
                channels = waveFile.info.channels
                samplerate = waveFile.info.sample_rate
                date = "-"
                track = "-"
                composer = "-"
                encoder = "-"
                
                # append new dataFrame to initial dataFrame
                data = [artist, album, genre, fileformat, duration, title, date, samplerate, bitrate, track, composer, encoder, channels]
                df2 = pd.DataFrame(data = [data],columns = columns)
                df = df.append(df2 ,ignore_index = True) 
            
            if(flag):
                
                # read file
                audioFile = EasyID3(fl)
                
                # add new row to dataFrame
                rows = np.append(rows, fl.name)
                
                # get metadata
                title = audioFile['title']
                artist = audioFile['artist']
                album = audioFile['album']
                genre = audioFile['genre']
                
                if isMpeg or isFlac:
                    # read file
                    af = MP3(fl)
                    duration = af.info.length/60
                 
                date = audioFile['date']
                samplerate = af.info.sample_rate
                bitrate = af.info.bitrate
                track = audioFile['tracknumber']
                composer = audioFile['composer']
                encoder = audioFile['encodedby']
                channels = af.info.channels      
                
                # append new dataFrame to initial dataFrame
                data = [artist, album, genre, fileformat, duration, title, date, samplerate, bitrate, track, composer, encoder, channels]
                df2 = pd.DataFrame(data = [data],columns = columns)
                df = df.append(df2 ,ignore_index = True) 
    
    # set file names as index
    df.index = rows
    
    # create tuple for the groupby function
    df[str(grouping.name)] = df[str(grouping.name)].apply(tuple)
    
    group = df.groupby(str(grouping.name))
        
    # sort values
    #toRet = df.sort_values(by=[str(grouping.name)])
    
    #return group.first()
    
    #return toRet
    
    return df
    

In [1089]:
# Test your function here.
class Criteria(Enum):
    artist = 1,
    album = 2,
    genre = 3
#MyAudioFilesOrganizer("./media/audio/Task2.1/*", Criteria.artist)


,artist,album,genre,format,duration,title,date,sample rate,bitrate,track,composer,encoder,channels
./media/audio/Task2.1/FireFire.mp3,"(M.I.A.,)",[Arular],[Hip Hop/Rap],mp3,3.480381,"[Fire, Fire]",[2005],44100,160000,[5/13],"[Maya Arulpragasam, Anthony Whiting]",[iTunes v7.1],2
./media/audio/Task2.1/Amazon.mp3,"(M.I.A.,)",[Arular],[Hip Hop/Rap],mp3,4.278857,[Amazon],[2005],44100,160000,[7/13],"[Maya Arulpragasam, Richard X.]",[iTunes v7.1],2
./media/audio/Task2.1/DashTheCurry[Skit].mp3,"(M.I.A.,)",[Arular],[Hip Hop/Rap],mp3,0.669605,[Dash The Curry [Skit]],[2005],44100,160000,[6/13],[Maya Arulpragasam],[iTunes v7.1],2
./media/audio/Task2.1/error.wav,"(-,)",-,-,wav,0.054422,-,-,44100,88200,-,-,-,1
./media/audio/Task2.1/Hombre.mp3,"(M.I.A.,)",[Arular],[Hip Hop/Rap],mp3,4.035482,[Hombre],[2005],44100,160000,[9/13],"[Maya Arulpragasam, Anthony Whiting]",[iTunes v7.1],2


## Task 2.2 Audio mixer (25P):

In [49]:
# Write your function here.

# Cuts an audio file by delegating the cutting to FFMPEG.
def TwoAudioMixer(audioFile1: str, a1From: int, a1To: int, 
                  audioFile2: str, a2From: int, a2To: int, overlapDur: float, 
                  outputDir: str, outFilename: str) -> None:
    
    # read input files
    input1 = ffmpeg.input(audioFile1)
        
    input2 = ffmpeg.input(audioFile2)
    
    # trim input files
    trim1 = input1.audio.filter("atrim", start = a1From, end = a1To)
       
    audioFile1Trimmed = "media/audio/Task2.2/out1.mp3"
    
    # save trimmed file
    ffmpeg.output(trim1, audioFile1Trimmed).run(overwrite_output=True)
    
    trim2 = input2.audio.filter("atrim", start = a2From, end = a2To)
    
    audioFile2Trimmed = "media/audio/Task2.2/out2.mp3"
    
    # save trimmed file
    ffmpeg.output(trim2, audioFile2Trimmed).run(overwrite_output=True)    
    
    # find duration of the first file 
    duration1 = a1To-a1From

    # timestamp where the second file should start for adelay
    time = duration1-overlapDur

    # output file name
    outputName = outputDir + outFilename
    
    # run ffmpeg terminal command
    subprocess.run("ffmpeg -y -i "+ audioFile1Trimmed +" -i "+ audioFile2Trimmed +" -filter_complex \"[1:a] adelay = "+str(time)+"s|"+str(time)+"s [delayoutput]; [0:a][delayoutput] amix [amixoutput]\" -map \"[amixoutput]\" " + outputName, shell=True)

In [51]:
# Test your function here.
#TwoAudioMixer("media/audio/Task2.2/FireFire.mp3", 0, 10, "media/audio/Task2.2/DashTheCurry[Skit].mp3", 0, 15, 5, "media/audio/Task2.2/", "outputMix.mp3")

## Task 2.3 Concealing speakers ID by lowering/increasing the audio pitch (20P):

In [7]:
# Write your function here.

# Generates a pandas DataFrame of audio metadata.
def VoicePitchChanger(audioFile: str, shift: float, outputDir: str, outFilename: str) -> None:
    
    # output file name
    outputName = outputDir + outFilename
    
    # run ffmpeg terminal command
    # https://hhsprings.bitbucket.io/docs/programming/examples/ffmpeg/manipulating_audio/atempo_asetrate_aresample.html
    subprocess.run("ffmpeg -y -i "+ audioFile + " -af \"asetrate = 44100*" + str(shift) + ", atempo = 1/" + str(shift) + ", aresample = 44100\" " + outputName, shell = True)
    

In [6]:
# Test your function here.
#VoicePitchChanger("media/audio/Task2.3/Hombre.mp3", 1.5, "media/audio/Task2.3/", "pitch.mp3")
